In [1]:
s='''
        aliceblue, antiquewhite, aqua, aquamarine, azure,
        beige, bisque, blanchedalmond, blue,
        blueviolet, brown, burlywood, cadetblue,
        chartreuse, chocolate, coral, cornflowerblue,
        cornsilk, crimson, cyan, darkblue, darkcyan,
        darkgoldenrod, darkgray, darkgrey, darkgreen,
        darkkhaki, darkmagenta, darkolivegreen, darkorange,
        darkorchid, darkred, darksalmon, darkseagreen,
        darkslateblue, darkslategray, darkslategrey,
        darkturquoise, darkviolet, deeppink, deepskyblue,
        dimgray, dimgrey, dodgerblue, firebrick,
        floralwhite, forestgreen, fuchsia, gainsboro,
        ghostwhite, gold, goldenrod, gray, grey, green,
        greenyellow, honeydew, hotpink, indianred, indigo,
        ivory, khaki, lavender, lavenderblush, lawngreen,
        lemonchiffon, lightblue, lightcoral, lightcyan,
        lightgoldenrodyellow, lightgray, lightgrey,
        lightgreen, lightpink, lightsalmon, lightseagreen,
        lightskyblue, lightslategray, lightslategrey,
        lightsteelblue, lightyellow, lime, limegreen,
        linen, magenta, maroon, mediumaquamarine,
        mediumblue, mediumorchid, mediumpurple,
        mediumseagreen, mediumslateblue, mediumspringgreen,
        mediumturquoise, mediumvioletred, midnightblue,
        mintcream, mistyrose, moccasin, navajowhite, navy,
        oldlace, olive, olivedrab, orange, orangered,
        orchid, palegoldenrod, palegreen, paleturquoise,
        palevioletred, papayawhip, peachpuff, peru, pink,
        plum, powderblue, purple, red, rosybrown,
        royalblue, saddlebrown, salmon, sandybrown,
        seagreen, seashell, sienna, silver, skyblue,
        slateblue, slategray, slategrey, snow, springgreen,
        steelblue, tan, teal, thistle, tomato, turquoise,
        violet, wheat, white, whitesmoke, yellow,
        yellowgreen
        '''
li=s.split(',')
li=[l.replace('\n','') for l in li]
li=[l.replace(' ','') for l in li]

import pandas as pd
import numpy as np
import math
import itertools
import copy as cp
from pulp import *
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages 

In [2]:
def initial_row(data,H,W,indices):
    l=0
    vertices=[]
    chosen=[]
    deleting_data=[]
    first_in_row=True
    x_center=[]
    y_center=[]
    weights=[]

    deleting_data=[]
    for i in range(0,len(data)):
        if l+data[i][1]<= W and  data[i][0]<=H:
            if first_in_row==True:
                x_0=0
                y_0=0
                x_1=data[i][1]
                y_1=data[i][0]
                chosen.append(indices[i])
                deleting_data.append(i)
                weights.append(data[i][0]*data[i][1])
                l=l+data[i][1]
                first_in_row=False
            else:
                x_0=vertices[-2]
                y_0=0
                x_1=x_0+data[i][1]
                y_1=data[i][0]
                chosen.append(indices[i])    
                deleting_data.append(i)
                weights.append(data[i][0]*data[i][1])
                l=l+data[i][1]
            vertices.append(x_0)
            vertices.append(y_0)
            vertices.append(x_1)
            vertices.append(y_1)
            x_center.append((vertices[-4]+vertices[-2])/2)
            y_center.append((vertices[-3]+vertices[-1])/2)
    if len(deleting_data)!=0:                    
        keep=[x for x in range(0,len(data)) if x not in deleting_data]
        indices=[indices[i] for i in keep]
        data=data[keep,:]

    return vertices, chosen, data, indices, x_center, y_center,weights

In [3]:
##right to left row operations.  starts with right edge at edge of bin and populates as close as possible to opposite
##edge of the bin
def right_to_left(data,chosen,H,W,indices,vertices,x_center,y_center,weights): 

    l=0
    deleting_data=[]
    first_in_row=True

     ##check to see if data was mistakenly input when there is none left in set to choose from
    if data.shape[0]==0:
        return vertices, chosen, data, indices, x_center, y_center,weights

    deleting_data=[]
    ##forming previous vertices into an n by 4 matrix for better indexing
    to_pick=np.reshape(vertices,(int(len(vertices)/4),4)) 

    ##first for each row is selected here.  
    for i in range(0,data.shape[0]):
        if l+data[i][1]<=W :
                ##if first_in_row==True:
                    x_0=W-data[i][1]
                    x_1=W

                    ##indexes maximum y-value in case the previous row does not sit flush with edge of bin.
                    indeces_for_test=[j for j in range(0,to_pick.shape[0]) if ((x_0<to_pick[j][0]<x_1) or
                                                                       (x_0<to_pick[j][2]<x_1) or
                                                                       (x_0<to_pick[j][0] and x_1>to_pick[j][2]) or
                                                                       (x_0>to_pick[j][0] and x_1<to_pick[j][2]) or
                                                                       (to_pick[j][0]<x_0<to_pick[j][2]) or
                                                                       (to_pick[j][0]<x_1<to_pick[j][2]) or
                                                                       (to_pick[j][0]<=x_0 and to_pick[j][2]>=x_1) or
                                                                        (to_pick[j][0]==x_0 and to_pick[j][2]==x_1))]
                    loop_index=to_pick[indeces_for_test,:]
                    ##conditions for addition of first item
                    if len(loop_index)>0:

                        y_0=max(loop_index[:,3])
                        if y_0+data[i][0]<=H:

                            y_1=y_0+data[i][0]
                            vertices.append(x_0)
                            vertices.append(y_0)
                            vertices.append(x_1)
                            vertices.append(y_1)
                            chosen.append(indices[i])    
                            deleting_data.append(i)
                            weights.append(data[i][0]*data[i][1])
                            l=l+data[i][1]
                            x_center.append((vertices[-4]+vertices[-2])/2)
                            y_center.append((vertices[-3]+vertices[-1])/2)
                            break
                            ##first_in_row=False

    if len(deleting_data)==0 or data.shape[0]==0:
        return vertices, chosen, data, indices, x_center, y_center,weights
    else:
        keep=[x for x in range(0,data.shape[0]) if x not in deleting_data]
        indices=[indices[i] for i in keep]
        data=data[keep,:]
        deleting_data=[]

    for i in range(0,data.shape[0]):

        if l+data[i][1] <=W :  

            x_0=vertices[-4]-data[i][1]
            x_1=vertices[-4]
            indeces_for_test=[j for j in range(0,to_pick.shape[0]) if ((x_0<to_pick[j][0]<x_1) or
                                                                       (x_0<to_pick[j][2]<x_1) or
                                                                       (x_0<to_pick[j][0] and x_1>to_pick[j][2]) or
                                                                       (x_0>to_pick[j][0] and x_1<to_pick[j][2]) or
                                                                       (to_pick[j][0]<x_0<to_pick[j][2]) or
                                                                       (to_pick[j][0]<x_1<to_pick[j][2]) or
                                                                       (to_pick[j][0]<=x_0 and to_pick[j][2]>=x_1) or 
                                                                        (to_pick[j][0]==x_0 and to_pick[j][2]==x_1))]
            loop_index=to_pick[indeces_for_test,:]

            if len(loop_index)>0:

                y_x=max(loop_index[:,3])

                if y_x+data[i][0]<=H:

                    y_0=y_x
                    y_1=y_x+data[i][0]

                    if y_0<vertices[-1] and y_1>vertices[-3]:

                        vertices.append(x_0)
                        vertices.append(y_0)
                        vertices.append(x_1)
                        vertices.append(y_1)
                        chosen.append(indices[i])    
                        deleting_data.append(i)
                        weights.append(data[i][0]*data[i][1])
                        l=l+data[i][1]
                        x_center.append((vertices[-4]+vertices[-2])/2)
                        y_center.append((vertices[-3]+vertices[-1])/2)
                        ##allows the items to climb or lower
                    elif vertices[-3]-y_x>0 and vertices[-3]-y_x>=data[i][0]:
                        if vertices[-3]+data[i][0]<=H:

                            y_0=vertices[-3]
                            y_1=y_0+data[i][0]
                            vertices.append(x_0)
                            vertices.append(y_0)
                            vertices.append(x_1)
                            vertices.append(y_1)
                            chosen.append(indices[i])    
                            deleting_data.append(i)
                            weights.append(data[i][0]*data[i][1])
                            l=l+data[i][1]
                            x_center.append((vertices[-4]+vertices[-2])/2)
                            y_center.append((vertices[-3]+vertices[-1])/2)  

    if len(deleting_data)>0: 
        keep=[x for x in range(0,data.shape[0]) if x not in deleting_data]
        indices=[indices[i] for i in keep]
        data=data[keep,:]

    return vertices, chosen, data, indices, x_center, y_center, weights

In [4]:
def left_to_right(data,chosen,H,W,indices,vertices,x_center,y_center,weights):

    l=0
    deleting_data=[]
    first_in_row=True

    ##check to see if data was mistakenly input when there is none left in set to choose from
    if data.shape[0]==0:
        return vertices, chosen, data, indices, x_center, y_center,weights

    deleting_data=[]
    ##forming previous vertices into an n by 4 matrix for better indexing
    to_pick=np.reshape(vertices,(int(len(vertices)/4),4))

    for i in range(0,data.shape[0]):

        if l+data[i][1]<=W :

                    x_0=0
                    x_1=data[i][1]
                    indeces_for_test=[j for j in range(0,to_pick.shape[0]) if ((x_0<to_pick[j][0]<x_1) or
                                                                       (x_0<to_pick[j][2]<x_1) or
                                                                       (x_0<to_pick[j][0] and x_1>to_pick[j][2]) or
                                                                       (x_0>to_pick[j][0] and x_1<to_pick[j][2]) or
                                                                       (to_pick[j][0]<x_0<to_pick[j][2]) or
                                                                       (to_pick[j][0]<x_1<to_pick[j][2]) or
                                                                       (to_pick[j][0]<=x_0 and to_pick[j][2]>=x_1) or 
                                                                        (to_pick[j][0]==x_0 and to_pick[j][2]==x_1))]
                    loop_index=to_pick[indeces_for_test,:]
                    ##conditions for addition of first item
                    if len(loop_index)>0:
                        y_0=max(loop_index[:,3])

                        if y_0+data[i][0]<=H:

                            y_1=y_0+data[i][0] 
                            vertices.append(x_0)
                            vertices.append(y_0)
                            vertices.append(x_1)
                            vertices.append(y_1)
                            chosen.append(indices[i])    
                            deleting_data.append(i)
                            weights.append(data[i][0]*data[i][1])
                            l=l+data[i][1]
                            x_center.append((vertices[-4]+vertices[-2])/2)
                            y_center.append((vertices[-3]+vertices[-1])/2)
                            break

    if len(deleting_data)==0 or data.shape[0]==0:
        return vertices, chosen, data, indices, x_center, y_center, weights
    else:
        keep=[x for x in range(0,data.shape[0]) if x not in deleting_data]
        indices=[indices[i] for i in keep]
        data=data[keep,:]
        deleting_data=[]

    for i in range(0,data.shape[0]):

        if l+data[i][1] <=W :

            x_0=vertices[-2]
            x_1=vertices[-2]+data[i][1]
            indeces_for_test=[j for j in range(0,to_pick.shape[0]) if ((x_0<to_pick[j][0]<x_1) or
                                                                       (x_0<to_pick[j][2]<x_1) or
                                                                       (x_0<to_pick[j][0] and x_1>to_pick[j][2]) or
                                                                       (x_0>to_pick[j][0] and x_1<to_pick[j][2]) or
                                                                       (to_pick[j][0]<x_0<to_pick[j][2]) or
                                                                       (to_pick[j][0]<x_1<to_pick[j][2]) or
                                                                       (to_pick[j][0]<=x_0 and to_pick[j][2]>=x_1) or 
                                                                        (to_pick[j][0]==x_0 and to_pick[j][2]==x_1))]
            loop_index=to_pick[indeces_for_test,:]
            ##loop to check height and width condition
            if len(loop_index)>0:

                y_x=max(loop_index[:,3])

                if y_x+data[i][0]<=H:

                    y_0=y_x
                    y_1=y_x+data[i][0]

                    if y_0<vertices[-1] and y_1>vertices[-3] :

                        vertices.append(x_0)
                        vertices.append(y_0)
                        vertices.append(x_1)
                        vertices.append(y_1)
                        chosen.append(indices[i])    
                        deleting_data.append(i)
                        weights.append(data[i][0]*data[i][1])
                        l=l+data[i][1]
                        x_center.append((vertices[-4]+vertices[-2])/2)
                        y_center.append((vertices[-3]+vertices[-1])/2)
                        ##allows the items to climb or lower
                    elif vertices[-3]-y_x>0 and vertices[-3]-y_x>=data[i][0]:
                        if vertices[-3]+data[i][0]<=H:
                            y_0=vertices[-3]
                            y_1=y_0+data[i][0]
                            vertices.append(x_0)
                            vertices.append(y_0)
                            vertices.append(x_1)
                            vertices.append(y_1)
                            chosen.append(indices[i])    
                            deleting_data.append(i)
                            weights.append(data[i][0]*data[i][1])
                            l=l+data[i][1]
                            x_center.append((vertices[-4]+vertices[-2])/2)
                            y_center.append((vertices[-3]+vertices[-1])/2)

    if len(deleting_data)>0:                    
        keep=[x for x in range(0,data.shape[0]) if x not in deleting_data]
        indices=[indices[i] for i in keep]
        data=data[keep,:]

    return vertices, chosen, data, indices, x_center, y_center,weights

In [5]:
def clustering_H_A(data):

    cluster_indices_height=[]
    total_clusters_height=[]
    cluster_indices_area=[]
    total_clusters_area=[]
    
    items=len(data)
    beta=0.75
    h_first_in_cluster=0
    a_first_in_cluster=0
    ##loop here to stop when all items are added to cluster
    h_chosen_amnt=0
    a_chosen_amnt=0
    
    while h_chosen_amnt<items:
        ##indexing where the specified height concentration is met
        h_data_insert=data[(np.where(data[h_first_in_cluster:,0]>=(beta*data[h_first_in_cluster][0]))[0])+h_first_in_cluster,:]
        ##sorting in descending order
        h_data_insert_height=h_data_insert[h_data_insert[:, 1].argsort()[::-1]]
        ##heights array for 1D bin packing
        total_clusters_height.append(h_data_insert_height)
        ##indices for indexing for graph
        cluster_indices_height.append(h_data_insert_height[:,4])
        ##incrementing amount chosen for layer for loop condition
        h_chosen_amnt=h_chosen_amnt+len(h_data_insert)
        ##next spot for the incrementation.  no subtraction necessary because it starts at the next consecutive spot
        h_first_in_cluster=h_chosen_amnt
        
    while a_chosen_amnt<items:
        ##indexing where the specified area concentration is met
        a_data_insert=data[(np.where(data[a_first_in_cluster:,4]>=(beta*data[a_first_in_cluster][4]))[0])+a_first_in_cluster,:]
        ##sorting in descending order
        a_data_insert_area=a_data_insert[a_data_insert[:, 1].argsort()[::-1]]
        ##heights array for 1D bin packing
        total_clusters_area.append(a_data_insert_area)
        ##indices for indexing for graph
        cluster_indices_area.append(a_data_insert_area[:,4])
        
        a_chosen_amnt=a_chosen_amnt+len(a_data_insert)
        a_first_in_cluster=a_chosen_amnt
        
    return cluster_indices_height, total_clusters_height, cluster_indices_area, total_clusters_area

In [6]:
def create_layers(data,indices):

    ##arrays for return
    cluster_vertices=[]
    cluster_chosen=[]
    cluster_w_center=[]
    cluster_d_center=[]
    height_layers=[]

    for i in range(0,len(data)):
        ##arrays for minimization and graphing
        bins_vertices=[]
        bins_chosen=[]
        bins_centers_x=[]
        bins_centers_y=[]
        max_bin_height=[]

        d=cp.deepcopy(indices[i])
        c=cp.deepcopy(data[i][:,1:3])
        heights_number=cp.deepcopy(data[i][:,[0,4]])
        ##continue to create new bins until there are no more items left
        while len(c)>1:

            ##explicitly entered bin dimensions
            a,b,c,d,e,f,g=initial_row(c,30,30,d)

            rl=0
            lr=len(a)
            between=0
            ##loop ending conditions where consecutive runs don't input any items into the present bin.  
            while (rl==between==lr)==False:
                rl=len(c)

                a,b,c,d,e,f,g=right_to_left(c,b,30,30,d,a,e,f,g)
                between=len(c)

                a,b,c,d,e,f,g=left_to_right(c,b,30,30,d,a,e,f,g)
                lr=len(c)
            ##index data where chosen is found and take max height 
            height_indices=np.arange(heights_number[:,1].shape[0])[np.in1d(heights_number[:,1],b)]
            ##maximum of all heights of the layer.  will need to be this height to fit.  
            max_bin_height.append(max(heights_number[height_indices,0]))  
            ##after each bin-stopping condition is met the parameters are entered into corresponding arrays
            bins_vertices.append(a)
            bins_chosen.append(b)
            bins_centers_x.append(e)
            bins_centers_y.append(f)  

        height_layers.append(max_bin_height)
        cluster_vertices.append(bins_vertices)
        cluster_chosen.append(bins_chosen)
        cluster_w_center.append(bins_centers_x)
        cluster_d_center.append(bins_centers_y)

    return height_layers,cluster_vertices,cluster_chosen,cluster_w_center,cluster_d_center

In [7]:
def bin_placement(layer_heights,max_height):

    ##total layers
    bin_layers=[]
    ##indices from flattened list.  easier to graph and separate later on.  additionally controls assignment
    bin_indices=[]
    ##number of bins according to heights.  flattened array
    height_list = [item for sublist in layer_heights for item in sublist]
    ##getting index name for each item for 1d bin problem
    list_string = np.arange(0,len(height_list))
    ##tuble of label with height ex: ('0',10)
    items=tuple(zip(list_string,height_list))
    
    ##maximum number of bins.  set to amount of items which is worst case scenario.  all could be capacity height
    U=len(items)
    ##amount of items
    itemCount=U
    ##capacity
    C=max_height
    ##bin variable
    Y = pulp.LpVariable.dicts('Bin_Used', range(U),
                                lowBound = 0,
                                upBound = 1,
                                cat = 'Integer')
    #allows for assigning of a variable
    possible_ItemInBin = [(itemTuple[0], binNum) for itemTuple in items
                                            for binNum in range(0,U)]
    
    ##kept as a dictionary to better index each separate bin
    X = pulp.LpVariable.dicts('itemInBin', possible_ItemInBin,
                            lowBound = 0,
                            upBound = 1,
                            cat = 'Integer')
    
    # Initialize the problem
    prob = LpProblem("Bin_Packing_Problem", LpMinimize)
    # Add the objective function.
    prob += lpSum([Y[i] for i in range(U)])
    ##constraint to have each item used once
    for j in items:
        prob += lpSum([X[(j[0], i)] for i in range(0,U)]) == 1
    ##constraint for the amount of items in each bin used to not exceed capacity
    for i in range(U):
        prob += lpSum([items[j][1] * X[(items[j][0], i)] for j in range(0,itemCount)]) <= C*Y[i]
    
    ##option to save
    ##prob.writeLP("BinPack.lp")
    prob.solve()
    ##dictionary with bin assignments
    bins = {}
    for item in X.keys():
        if X[item].value() ==1 :
            itemNum = item[0]
            binNum = item[1]
            if binNum in bins:
                bins[binNum].append(itemNum)
            else:
                bins[binNum] = [itemNum]
                
    ##assigning consecutive integers for bin numbers as opt. program can take random bins
    new_keys = list(map(str, range(0,len(bins.keys()))))
    ##sorting by key
    bins = dict(zip(new_keys, list(bins.values())))          
    
    return bins,items

In [8]:
def cluster_from_assignment(assignments,all_heights,all_vertices,all_chosen,all_widths,all_depths):
    
    ##flattened lists to better index  by assignments
    flat_height=[item for sublist in all_heights for item in sublist]
    flat_clust_verts=[item for sublist in all_vertices for item in sublist]
    flat_clust_chosen=[item for sublist in all_chosen for item in sublist]
    flat_clust_width=[item for sublist in all_widths for item in sublist]
    flat_clust_depth=[item for sublist in all_depths for item in sublist]
    
    ##empty lists for separate bins
    bin_height=[]
    bin_clust_verts=[]
    bin_clust_chosen=[]
    bin_clust_width=[]
    bin_clust_depth=[]
    
    ##indexing the specific values for the layer input for each separate bin.  only have the keys and values from 
    ##1D Bin packing
    for item in assignments.values():
        loop_height=[]
        loop_clust_v=[]
        loop_clust_chose=[]
        loop_clust_w=[]
        loop_clust_d=[]
        ##get past indexing data
        pick_list=[i for i in item]
        
        for j in pick_list:
            loop_height.append(flat_height[j])
            loop_clust_v.append(flat_clust_verts[j])
            loop_clust_chose.append(flat_clust_chosen[j])
            loop_clust_w.append(flat_clust_width[j])
            loop_clust_d.append(flat_clust_depth[j])
        
        bin_height.append(loop_height)
        bin_clust_verts.append(loop_clust_v)
        bin_clust_chosen.append(loop_clust_chose)
        bin_clust_width.append(loop_clust_w)
        bin_clust_depth.append(loop_clust_d)
    
    return bin_height, bin_clust_verts, bin_clust_chosen, bin_clust_width, bin_clust_depth

In [9]:
def choose_min_bin(height_info,area_info):
    
    ##amount of height bins
    h_bins=len(height_info[0])
    ##amount of area bins
    a_bins=len(area_info[0])
    
    ##choosing the max of each
    if h_bins<=a_bins:
        return height_info
    else:
        return area_info

In [10]:
def save_image(filename):
    
        # PdfPages is a wrapper around pdf 
        # file so there is no clash and
        # create files with no error.
        p = PdfPages(filename)

        # get_fignums Return list of existing
        # figure numbers
        fig_nums = plt.get_fignums()  
        figs = [plt.figure(n) for n in fig_nums]

        # iterating over the numbers in list
        for fig in figs: 

            # and saving the files
            fig.savefig(p, format='pdf') 
            plt.close()
        # close the object
        p.close()  

In [11]:
def save_image(filename):
    
        p = PdfPages(filename)
        fig_nums = plt.get_fignums()  
        figs = [plt.figure(n) for n in fig_nums]

        # iterating over the numbers in list
        for fig in figs: 

            # and saving the files
            fig.savefig(p, format='pdf') 
            plt.close()
        # close the object
        p.close()  

In [12]:
def graph_bin_by_layers(adv_info,dims_layer,fill_col):
    
    ##loop by length of bins
    for i in range(0,len(adv_info[0])):
        ##loop by length of layers in each bin
        for j in range(0,len(adv_info[2][i])):
            
            fig = plt.figure(figsize=(10,10))
            ax = fig.add_subplot(111)
            plt.xlim([0, dims_layer[0]])
            plt.ylim([0, dims_layer[1]])
            ##color array to repeat and not over index.  not random but goes down list
            color_for_loop=fill_col*math.ceil(len(adv_info[2][i][j])/len(fill_col))
            ##loop by length of indices to index the vertices to have origin point and width and depth
            for k in range(0,len(adv_info[2][i][j])):
                ##plotting with the vertices and width*depth as x*y.  height is taken care of by layer arrangement
                rect = matplotlib.patches.Rectangle((adv_info[1][i][j][k*4], adv_info[1][i][j][k*4+1]),
                                                     adv_info[1][i][j][k*4+2]-adv_info[1][i][j][k*4], adv_info[1][i][j][k*4+3]-adv_info[1][i][j][k*4+1],
                                                     facecolor =color_for_loop[k],edgecolor='black',)

                ax.add_patch(rect)
                ##adding the label and the outline
                ax.annotate(str(adv_info[2][i][j][k]), (adv_info[3][i][j][k], adv_info[4][i][j][k]), color='black', weight='heavy', fontsize=10, ha='center', va='center')
                ##naming accoring to bin and layer number.  saving multiple for each file
                ax.set_title(("Bin %d, Layer %d" % (i, j)))
        
        ##name by bin number and layer
        filename = str("Bin %d" % (i) )

        #saving each layer that is in each bin
        save_image(filename)  
        plt.close()

In [13]:
##generating random dataset for testing
df = pd.DataFrame(np.random.randint(1,10,size=(300, 3)), columns=["Height","Width","Depth"])
##column to matrix
df["Area"]=df[["Width"]].multiply(df["Depth"], axis="index")
df["index"]=np.array(df.index.to_numpy())
##dataframe sorting.  depends on either height first or area first
df=df.sort_values(by =['Height'], ascending = [False], na_position = 'first')
##indexing dataframe                          
d=np.array(df.index.to_numpy())
##dataframe to matrix 
c=df.to_numpy()

layer_dimensions=[30,30]
height_of_bin=10

##clustering both height and area
indices_out_height, clusters_out_height, indices_out_area, clusters_out_area=clustering_H_A(c)

##creating layers for the bins
h_heights,h_clust_verts,h_clust_chosen,h_cluster_width_c,h_cluster_depth_c=create_layers(clusters_out_height,indices_out_height)
a_heights,a_clust_verts,a_clust_chosen,a_cluster_width_c,a_cluster_depth_c=create_layers(clusters_out_area,indices_out_area)

##1D bin packing to assign layers to bins by height and area
h_bins_assignments,h_items=bin_placement(h_heights,height_of_bin)
a_bins_assignments,a_items=bin_placement(a_heights,height_of_bin)

##assigning the data based off of bin assignment
h_assigned_height,h_assigned_vertices,h_assigned_indices,h_assigned_widths,h_assigned_depths=cluster_from_assignment(h_bins_assignments,h_heights,h_clust_verts,h_clust_chosen,h_cluster_width_c,h_cluster_depth_c)
a_assigned_height,a_assigned_vertices,a_assigned_indices,a_assigned_widths,a_assigned_depths=cluster_from_assignment(a_bins_assignments,a_heights,a_clust_verts,a_clust_chosen,a_cluster_width_c,a_cluster_depth_c)

##total height array
height_total=[]

##adding the data for height
height_total.append(h_assigned_height)
height_total.append(h_assigned_vertices)
height_total.append(h_assigned_indices)
height_total.append(h_assigned_widths)
height_total.append(h_assigned_depths)


##total area array
area_total=[]

##adding the data for area
area_total.append(a_assigned_height)
area_total.append(a_assigned_vertices)
area_total.append(a_assigned_indices)
area_total.append(a_assigned_widths)
area_total.append(a_assigned_depths)

##choosing the max between sorted height or area
advantageous_sort=choose_min_bin(height_total,area_total)

##saving the graphs for one file for each bin
graph_bin_by_layers(advantageous_sort,layer_dimensions,li)